# 1.6 signac-flow HOOMD-blue Example

## About

This notebook contains a minimal example for running a *signac-flow* project from scratch.
The example demonstrates how to compare an ideal gas with a Lennard-Jones (LJ) fluid by calculating a p-V phase diagram.

This examples uses the general-purpose simulation toolkit [HOOMD-blue](http://glotzerlab.engin.umich.edu/hoomd-blue/) for the execution of the molecular-dynamics (MD) simulations.

## Author

Carl Simon Adorf, Bradley Dice

## Before you start

This example requires signac, signac-flow, HOOMD-blue and numpy.
You can install these package for example via conda:
```
conda install -c conda-forge hoomd numpy signac signac-flow
```

In [1]:
import itertools
import os

import flow
import gsd.hoomd
import hoomd
import numpy as np
import signac

project_path = "projects/tutorial-signac-flow-hoomd"


class MyProject(flow.FlowProject):
    pass

/home/brandon/mambaforge/envs/research/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We want to generate a pressure-volume phase diagram for a Lennard-Jones fluid with molecular dynamics (MD) using the general-purpose simulation toolkit HOOMD-blue (http://glotzerlab.engin.umich.edu/hoomd-blue/).

We start by defining two functions, one for the initialization of our simulation and one for the actual execution.

In [2]:
from math import ceil


def init(N):
    frame = gsd.hoomd.Frame()
    frame.particles.N = N
    frame.particles.types = ["A"]
    frame.particles.typeid = np.zeros(N)

    n = ceil(pow(N, 1 / 3))
    assert n**3 == N
    spacing = 1.2
    L = n * spacing
    frame.configuration.box = [L, L, L, 0.0, 0.0, 0.0]
    x = np.linspace(-L / 2, L / 2, n, endpoint=False)
    position = list(itertools.product(x, repeat=3))
    frame.particles.position = position
    with gsd.hoomd.open("init.gsd", "wb") as traj:
        traj.append(frame)


def get_sim(N, sigma, seed, kT, tau, p, tauP, r_cut):
    sim = hoomd.Simulation(hoomd.device.CPU())
    state_fn = "init.gsd"
    if os.path.exists("restart.gsd"):
        state_fn = "restart.gsd"
    sim.create_state_from_gsd(state_fn)
    sim.operations += hoomd.write.GSD(
        filename="restart.gsd", truncate=True, trigger=100, filter=hoomd.filter.All()
    )
    lj = hoomd.md.pair.LJ(default_r_cut=r_cut, nlist=hoomd.md.nlist.Cell(0.4))
    lj.params[("A", "A")] = {"epsilon": 1.0, "sigma": sigma}
    integrator = hoomd.md.Integrator(
        dt=0.005,
        methods=[
            hoomd.md.methods.NPT(
                filter=hoomd.filter.All(), kT=kT, tau=tau, S=p, tauS=tauP, couple="xyz"
            )
        ],
        forces=[lj],
    )
    sim.operations.integrator = integrator
    logger = hoomd.logging.Logger(categories=("scalar", "string"))
    logger["Volume"] = (lambda: sim.state.box.volume, "scalar")
    table = hoomd.write.Table(trigger=100, logger=logger, max_header_len=20)
    sim.operations += table
    return sim

We want to use **signac** to manage our simulation data and **signac-flow** to define a workflow acting on the data space.

Now that we have defined the core simulation logic above, it's time to embed those functions into a general *workflow*.
For this purpose we add labels and operations with preconditions/postconditions to `MyProject`, a subclass of `flow.FlowProject`.

The `estimate` operation stores an ideal gas estimation of the volume for the given system.
The `sample` operation actually executes the MD simulation as defined in the previous cell.

In [3]:
@MyProject.label
def estimated(job):
    return "V" in job.document


@MyProject.label
def sampled(job):
    return job.document.get("sample_step", 0) >= 5000


@MyProject.post(estimated)
@MyProject.operation
def estimate(job):
    sp = job.statepoint()
    job.document["V"] = sp["N"] * sp["kT"] / sp["p"]


@MyProject.post(sampled)
@MyProject.operation
def sample(job):
    with job:
        try:
            sim = get_sim(**job.statepoint())
            sim.run(5_000)
        finally:
            job.document["sample_step"] = sim.timestep

Now it's time to actually generate some data! Let's initialize the data space!


In [4]:
project = MyProject.init_project()

# Uncomment the following two lines if you want to start over!
# for job in project:
#     job.remove()

for p in np.linspace(0.5, 5.0, 10):
    sp = dict(N=512, sigma=1.0, seed=42, kT=1.0, p=p, tau=1.0, tauP=1.0, r_cut=2.5)
    job = project.open_job(sp).init()
    with job:
        init(N=sp["N"])

/home/brandon/mambaforge/envs/research/lib/python3.11/site-packages/signac/synced_collections/numpy_utils.py:46: NumpyConversionWarning: Any numpy types provided will be transparently converted to the closest base Python equivalents.
  warnings.warn(NUMPY_CONVERSION_WARNING, NumpyConversionWarning)


The `print_status()` function allows to get a quick overview of our project's *status*:

In [5]:
project.print_status(detailed=True, parameters=["p"])

Fetching labels: 100%|██████████| 10/10 [00:00<00:00, 23327.61it/s]


Overview: 10 jobs/aggregates, 1 jobs/aggregates with eligible operations.

label      ratio
---------  --------------------------------------------------------
estimated  |████████████████████████████████████    | 9/10 (90.00%)
sampled    |████████████████████████████████████    | 9/10 (90.00%)

operation/group      number of eligible jobs  submission status
-----------------  -------------------------  -------------------
estimate                                   1  [U]: 10
sample                                     1  [U]: 10

Detailed View:

job id                            operation/group      p  labels
--------------------------------  -----------------  ---  ------------------
18c063206c37ab03d06de130c6ae2b70  estimate [U]       1
                                  sample [U]         1
2a05a487f254db90b08f576d199a83b7  [ ]                5    estimated, sampled
4ef567c185064a4b27b73f62124991c0  [ ]                3    estimated, sampled
5b3c8f0fbd9458a304222d83bb82bc30  [ ]    

The next cell will attempt to execute all eligible operations.

In [6]:
project.run()

/home/brandon/mambaforge/envs/research/lib/python3.11/site-packages/hoomd/md/methods/methods.py:365: FutureWarning: NPT is deprecated and wil be removed in hoomd 4.0. In version 4.0, use the ConstantPressure method with the desired thermostat from hoomd.md.methods.thermostats.
  warnings.warn(


     Volume     
   815.08231    
   647.05649    
   507.51113    
   559.16728    
   735.55354    
   865.93540    
   867.29199    
   739.72163    
   564.60452    
   476.05178    
   3613.70596   
   7678.21985   
   1811.69480   
   684.11459    
   566.85211    
   541.47357    
   530.95724    
   527.61614    
   523.46338    
   518.79266    
   511.26080    
   503.74124    
   497.58350    
   495.06338    
   496.35727    
   501.29643    
   508.69586    
   520.15390    
   551.45089    
   666.08444    
   1225.90832   
   2926.64495   
   2373.50483   
   1388.31905   
   821.54009    
   632.14399    
   568.96207    
   541.86632    
   529.13999    
   522.20249    
   518.53066    
   518.30696    
   520.59497    
   532.97952    
   566.27799    
   647.24362    
   856.46153    
   1308.43561   
   1713.72357   
   1420.21424   


Let's double check the project status.

In [10]:
project.print_status(detailed=True)

Fetching status: 0it [00:00, ?it/s]
Fetching labels: 100%|██████████| 10/10 [00:00<00:00, 19831.22it/s]


Overview: 0 jobs/aggregates, 0 jobs/aggregates with eligible operations.

label
-------


operation/group
-----------------


Detailed View:

job id
--------


[U]:unknown [R]:registered [I]:inactive [S]:submitted [H]:held [Q]:queued [A]:active [E]:error [GR]:group_registered [GI]:group_inactive [GS]:group_submitted [GH]:group_held [GQ]:group_queued [GA]:group_active [GE]:group_error




After running all operations we can make a brief examination of the collected data.

In [ ]:
def get_volume(job):
    log = np.genfromtxt(job.fn("dump.log"), names=True)
    N = len(log)
    return log[int(0.5 * N) :]["volume"].mean(axis=0)


for job in project:
    print(job.statepoint["p"], get_volume(job), job.document.get("V"))

For a better presentation of the results we need to aggregate all results and sort them by pressure.

*The following code requires matplotlib.*

In [ ]:
%matplotlib inline
# Display plots within the notebook

from matplotlib import pyplot as plt

V = dict()
V_idg = dict()

for job in project:
    V[job.statepoint()["p"]] = get_volume(job)
    V_idg[job.statepoint()["p"]] = job.document["V"]

p = sorted(V.keys())
V = [V[p_] for p_ in p]
V_idg = [V_idg[p_] for p_ in p]

plt.plot(p, V, label="LJ")
plt.plot(p, V_idg, label="idG")
plt.xlabel(r"pressure [$\epsilon / \sigma^3$]")
plt.ylabel(r"volume [$\sigma^3$]")
plt.legend()
plt.show()

Uncomment and execute the following line to remove all data and start over.

In [ ]:
# %rm -r projects/tutorial-signac-flow-hoomd-blue/workspace